In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import re
import math

In [26]:
orig_caml_train = pd.read_csv("caml_train.csv", header = [0])
orig_caml_train.head()

,Report ID,Text,Label
0,1,the cardiac silhouette and mediastinum size ar...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,10,the cardiomediastinal silhouette is within nor...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,100,both lungs are clear and expanded . heart and ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,1000,there is xxxx increased opacity within the rig...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
4,1002,status post left mastectomy . heart size norma...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [27]:
orig_caml_test = pd.read_csv("caml_test.csv", header = [0])
orig_caml_test.head()

,Report ID,Text,Label
0,1001,interstitial markings are diffusely prominent ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1007,trachea is midline . the cardiomediastinal sil...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,101,the heart is again mildly enlarged . mediastin...,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ..."
3,1012,lung volumes are low . bibasilar consolidation...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
4,1014,slightly enlarged heart . no effusions . no ed...,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [28]:
num_train = len(orig_caml_train)
num_test = len(orig_caml_test)
print(f'TRAIN: {num_train}, TEST: {num_test}')

TRAIN: 3141, TEST: 786


The above splits were the train and test for CAML data used in the original homework assignment. To replicate the CLINICALBERT study by Nwamake Imasogie, I am going to split these differently after creating a label based on the first item in the "label" field.

In [29]:
caml_notes_all = pd.concat([orig_caml_train, orig_caml_test])
caml_notes_all = caml_notes_all.sort_values(["Report ID"]).reset_index(drop=True)
caml_notes_all = caml_notes_all.rename(columns={'Report ID': 'SUBJECT ID', 'Text': 'TEXT', 'Label': 'VALUES'})
print(caml_notes_all.columns)
print(caml_notes_all.head())

Index(['SUBJECT ID', 'TEXT', 'VALUES'], dtype='object')
   SUBJECT ID                                               TEXT  \
0           1  the cardiac silhouette and mediastinum size ar...   
1           2  borderline cardiomegaly . midline sternotomy x...   
2           3  no displaced rib fractures pneumothorax or ple...   
3           4  there are diffuse bilateral interstitial and a...   
4           5  the cardiomediastinal silhouette and pulmonary...   

                                              VALUES  
0  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
2  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, ...  
4  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [30]:
# Capture the first value in CAML, which is the label.
caml_notes_all["LABEL"] = caml_notes_all.VALUES.apply(lambda x: int(x[1]))
print('done')

done


In [31]:
caml_notes_all.head()

,SUBJECT ID,TEXT,VALUES,LABEL
0,1,the cardiac silhouette and mediastinum size ar...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,2,borderline cardiomegaly . midline sternotomy x...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,3,no displaced rib fractures pneumothorax or ple...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,4,there are diffuse bilateral interstitial and a...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, ...",0
4,5,the cardiomediastinal silhouette and pulmonary...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [32]:
caml_notes_all = caml_notes_all.drop(columns = ["VALUES"])
caml_notes_all.head()

,SUBJECT ID,TEXT,LABEL
0,1,the cardiac silhouette and mediastinum size ar...,1
1,2,borderline cardiomegaly . midline sternotomy x...,0
2,3,no displaced rib fractures pneumothorax or ple...,1
3,4,there are diffuse bilateral interstitial and a...,0
4,5,the cardiomediastinal silhouette and pulmonary...,0


In [33]:
caml_notes_all["ADMITTIME"] = pd.to_datetime("2022-03-01")
caml_notes_all["DISCHTIME"] = pd.to_datetime("2022-04-15")
caml_notes_all.head()

,SUBJECT ID,TEXT,LABEL,ADMITTIME,DISCHTIME
0,1,the cardiac silhouette and mediastinum size ar...,1,2022-03-01,2022-04-15
1,2,borderline cardiomegaly . midline sternotomy x...,0,2022-03-01,2022-04-15
2,3,no displaced rib fractures pneumothorax or ple...,1,2022-03-01,2022-04-15
3,4,there are diffuse bilateral interstitial and a...,0,2022-03-01,2022-04-15
4,5,the cardiomediastinal silhouette and pulmonary...,0,2022-03-01,2022-04-15


In [34]:
caml_notes_all.loc[caml_notes_all["LABEL"] == 1, "DISCHTIME"] = pd.to_datetime("2022-03-15")
caml_notes_all.head()

,SUBJECT ID,TEXT,LABEL,ADMITTIME,DISCHTIME
0,1,the cardiac silhouette and mediastinum size ar...,1,2022-03-01,2022-03-15
1,2,borderline cardiomegaly . midline sternotomy x...,0,2022-03-01,2022-04-15
2,3,no displaced rib fractures pneumothorax or ple...,1,2022-03-01,2022-03-15
3,4,there are diffuse bilateral interstitial and a...,0,2022-03-01,2022-04-15
4,5,the cardiomediastinal silhouette and pulmonary...,0,2022-03-01,2022-04-15


The above caml notes have artificially set dates with readmission < 30 days for sick people and > 30 days for non-sick people. However the head TA does not think it's worth splitting he values based on artificially inserted dates (I was suggesting using a multinomial distribution).

The text records need to be cleaned as follows:
(a) make all words lowercase
(b) remove punctuation
(c) remove numbers and words that contain numbers
(d) remove newline characters \n and carriage returns \r
(e) tokenize the text
(f) remove stop words
(g) lemmatize the words (using part-of-speech tagging) so that the model performs better

## Tokenize

In [35]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import re
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer # lemmatizes word based on its parts of speechprint('Punctuation:', string.punctuation)

In [36]:
# Here are the stopwords
num_stopwords = len(stopwords.words('english'))
print(f'There are {num_stopwords} stopwords')
# print(stopwords.words('english'))

There are 153 stopwords


In [37]:
# The POS tags need to be convered so that they work in WordNet.
def convert_tag(treebank_tag):
    '''Convert Treebank tags to WordNet tags'''
    if treebank_tag.startswith('J'):
        return 'a'
    elif treebank_tag.startswith('V'):
        return 'v'
    elif treebank_tag.startswith('N'):
        return 'n'
    elif treebank_tag.startswith('R'):
        return 'r'
    else:
        return 'n' # if no match, default to noun

In [38]:
'''
This method takes tokens (a list of cleaned tokens with stopwords removed
and returns a list of lemma words (words that have been lemmatized).

    Params:
        tokens (list of strings): cleaned tokens with stopwords removed
        
    Return:
        lemma_words (list of strings): lemmatized words
''' 

def lemmatize(tokens):
 
    # POS-tag your data before lemmatizing
    tagged_words = pos_tag(tokens) # outputs list of tuples [('recent', 'JJ'),...]
    
    # Lemmatize using WordNet's built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
    wnl = WordNetLemmatizer()
    lemma_words = []
    
    # Lemmatize list of tuples, output a list of strings
    for tupl in tagged_words:
        lemma_words.append(wnl.lemmatize(tupl[0], convert_tag(tupl[1])))
    
    return lemma_words

In [39]:
'''
This preprocessing an tokenization method makes text lowercase, removes punctuation,
removes number and words that contain numbers, removes newline and carriage rturns,
tokenizes the words, removes stop words, and then lemmatizes the tokens and returns
the lemmatized words.

Params:
    text (string): full original, uncleaned text
Returns:
    lemmatized_tokens (list of strings): cleaned words
'''

def preprocess_and_tokenize(text):

    # Make text lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # Remove numbers and words that contain numbers
    text = re.sub('\w*\d\w*', '', text)
    
    # Remove newline chars and carriage returns
    text = re.sub('\n', '', text)
    text = re.sub('\r', '', text)
    
    # Tokenize
    word_tokens = word_tokenize(text) 
    
    # Remove stop words
    tokens = [word for word in word_tokens if word not in stopwords.words('english')]
    
    # Call the above lemmatizer function to perform lemmatization
    lemmatized_tokens = lemmatize(tokens)
    
    return lemmatized_tokens

In [40]:
# test_string = 'A list of words having 3 or 4 numbers and punctuation!'
test_string = caml_notes_all.TEXT.loc[0]
preprocess_and_tokenize(test_string)

['cardiac',
 'silhouette',
 'mediastinum',
 'size',
 'within',
 'normal',
 'limit',
 'pulmonary',
 'edema',
 'focal',
 'consolidation',
 'xxxx',
 'pleural',
 'effusion',
 'evidence',
 'pneumothorax',
 'normal',
 'chest',
 'xxxxx']

We should probably simply sanitize all of the data at once before splitting it into test / train / val sets. 
(The original article didn't do this.)

In [41]:
num_notes = len(caml_notes_all)
caml_notes_all["TEXT"] = caml_notes_all.TEXT.apply(preprocess_and_tokenize)
print('done')

done


In [42]:
print(len(caml_notes_all))
caml_notes_all.head()

3927


,SUBJECT ID,TEXT,LABEL,ADMITTIME,DISCHTIME
0,1,"[cardiac, silhouette, mediastinum, size, withi...",1,2022-03-01,2022-03-15
1,2,"[borderline, cardiomegaly, midline, sternotomy...",0,2022-03-01,2022-04-15
2,3,"[displace, rib, fracture, pneumothorax, pleura...",1,2022-03-01,2022-03-15
3,4,"[diffuse, bilateral, interstitial, alveolar, o...",0,2022-03-01,2022-04-15
4,5,"[cardiomediastinal, silhouette, pulmonary, vas...",0,2022-03-01,2022-04-15


In [43]:
caml_notes_all['TEXTSTRING'] = caml_notes_all['TEXT'].apply(' '.join)
print('done')

done


In [44]:
caml_notes_all.head()

,SUBJECT ID,TEXT,LABEL,ADMITTIME,DISCHTIME,TEXTSTRING
0,1,"[cardiac, silhouette, mediastinum, size, withi...",1,2022-03-01,2022-03-15,cardiac silhouette mediastinum size within nor...
1,2,"[borderline, cardiomegaly, midline, sternotomy...",0,2022-03-01,2022-04-15,borderline cardiomegaly midline sternotomy xxx...
2,3,"[displace, rib, fracture, pneumothorax, pleura...",1,2022-03-01,2022-03-15,displace rib fracture pneumothorax pleural eff...
3,4,"[diffuse, bilateral, interstitial, alveolar, o...",0,2022-03-01,2022-04-15,diffuse bilateral interstitial alveolar opacit...
4,5,"[cardiomediastinal, silhouette, pulmonary, vas...",0,2022-03-01,2022-04-15,cardiomediastinal silhouette pulmonary vascula...


In [45]:
max_seq_len = 0
num_recs = len(caml_notes_all)
print(num_recs)
for i in range(num_recs):
    this_len = len(caml_notes_all.TEXT.iloc[i])
    if this_len > max_seq_len:
        max_seq_len = this_len
print('Longest sequence is: ', max_seq_len)

3927
Longest sequence is:  138


#### Since the head TA Wu Zhenbang suggested I not use artificial dates, I am going to drop those and just keep TEXT and LABEL.

In [46]:
df_caml = caml_notes_all[["LABEL", "TEXTSTRING", "TEXT"]].copy()
df_caml = df_caml.rename(columns={'TEXT': 'TEXTLIST'})
df_caml.head()

,LABEL,TEXTSTRING,TEXTLIST
0,1,cardiac silhouette mediastinum size within nor...,"[cardiac, silhouette, mediastinum, size, withi..."
1,0,borderline cardiomegaly midline sternotomy xxx...,"[borderline, cardiomegaly, midline, sternotomy..."
2,1,displace rib fracture pneumothorax pleural eff...,"[displace, rib, fracture, pneumothorax, pleura..."
3,0,diffuse bilateral interstitial alveolar opacit...,"[diffuse, bilateral, interstitial, alveolar, o..."
4,0,cardiomediastinal silhouette pulmonary vascula...,"[cardiomediastinal, silhouette, pulmonary, vas..."


In [47]:
df_caml.to_csv('clean_caml.csv', index = False)

Next, we need a vectorizer to convert all this data to the equivalent of embeddings. It would be interesting to see how many words we have.

In [48]:
# DOUBLE CHECK

all_words = set()
for x in range(len(df_caml)):
    this_set = set(df_caml.TEXTLIST[x])
    all_words.update(this_set)
num_all_words = len(all_words)
print('Total unique words: ', num_all_words)

Total unique words:  1868


In [49]:
from sklearn.feature_extraction.text import CountVectorizer
her_vect = CountVectorizer(max_features = 3000, tokenizer = preprocess_and_tokenize)
print('done')

done


In [57]:
# For CAML we don't need so many features, and the tokenizer is not needed since the 
# data has already been processed
my_vect = CountVectorizer(max_features = num_all_words)
print('done')

done


In [58]:
df_caml.head()

,LABEL,TEXTSTRING,TEXTLIST
0,1,cardiac silhouette mediastinum size within nor...,"[cardiac, silhouette, mediastinum, size, withi..."
1,0,borderline cardiomegaly midline sternotomy xxx...,"[borderline, cardiomegaly, midline, sternotomy..."
2,1,displace rib fracture pneumothorax pleural eff...,"[displace, rib, fracture, pneumothorax, pleura..."
3,0,diffuse bilateral interstitial alveolar opacit...,"[diffuse, bilateral, interstitial, alveolar, o..."
4,0,cardiomediastinal silhouette pulmonary vascula...,"[cardiomediastinal, silhouette, pulmonary, vas..."


In [73]:
new_cv = CountVectorizer(df_caml.TEXTSTRING) # preprocessor = my_cool_preprocessor
my_vector = new_cv.fit_transform(df_caml.TEXTLIST.values)
# # df_caml.head()
new_cv.vocabulary_


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [63]:
# I applied .astype(str) to fix the ValueError: np.nan is an invalid document, expected byte or unicode string.xc
# create the vectorizer
df_caml.TEXT = df_caml.TEXTLIST.apply(my_vect.fit_transform)
print('done')

done


In [53]:
df_caml.TEXT.head()

0      (0, 0)\t1\n  (1, 13)\t1\n  (2, 8)\t1\n  (3, ...
1      (0, 2)\t1\n  (1, 3)\t1\n  (2, 9)\t1\n  (3, 1...
2      (0, 5)\t1\n  (1, 15)\t1\n  (2, 7)\t1\n  (3, ...
3      (0, 15)\t1\n  (1, 4)\t1\n  (2, 25)\t1\n  (3,...
4      (0, 4)\t1\n  (1, 22)\t1\n  (2, 21)\t1\n  (3,...
Name: TEXTLIST, dtype: object

In [ ]:
df_caml.TEXT.iloc[0]

In [ ]:
from sklearn.model_selection import train_test_split
X = caml_notes_all.TEXT
y = caml_notes_all.LABEL
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.20, random_state = 44)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = 0.50, random_state = 44)

In [ ]:
assert len(X_train) + len(X_test) + len(X_val) == len(caml_notes_all), 'Lengths incorrect'

In [ ]:
sum(caml_notes_all.LABEL) / len(caml_notes_all)

This CAML dataset is much less imbalanced than the MIMIC-III notes

In [ ]:
print("Training set prevalence (n = {:d}):".format(len(X_train)), "{:.2f}%".format((y_train.sum()/len(X_train))*100))
print("Validation set prevalence (n = {:d}):".format(len(X_val)), "{:.2f}%".format((y_val.sum()/len(X_val))*100))
print("Test set prevalence (n = {:d}):".format(len(X_test)), "{:.2f}%".format((y_test.sum()/len(X_test))*100))
print("All samples (n = {:d})".format(len(caml_notes_all)))

The first approach is to use Bag of Words. We first have to preprocess the text.

In [ ]:
# First train using the X_train vocabulary. You have to "unlist" each set, hence the lambda function.
my_vect.fit(X_train.apply(lambda x: ' '.join(x)))
print('done')

In [ ]:
len(my_vect.vocabulary_)

Earlier we saw that the total vocabulary was 1868, so there will be words that have not been trained for in the validation and test sets. We can create vectors by passing the text into the vectorizer to get back counts. We then sum over the columns to get the term frequencies.

In [ ]:
neg_doc_matrix = my_vect.transform(X_train[y_train == 0].astype(str))
pos_doc_matrix = my_vect.transform(X_train[y_train == 1].astype(str))

In [ ]:
neg_tf = np.sum(neg_doc_matrix, axis = 0)
pos_tf = np.sum(pos_doc_matrix, axis = 0)

In [ ]:
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))

We can convert the train and validation sets into numerical matrices as follows.

In [ ]:
X_train_tf = my_vect.transform(X_train.astype(str))
X_valid_tf = my_vect.transform(X_val.astype(str))

### Now we can build a simple predictive model

In [ ]:
X_train, y_train

In [ ]:
from sklearn.linear_model import LogisticRegression 
# Classifier
clf = LogisticRegression(C = 0.0001, penalty = 'l2', random_state = 44)
clf.fit(X_train_tf, y_train)
model = clf
print('done')

We now calculate the probability of being sick based on the model

In [ ]:
y_train_preds = model.predict_proba(X_train_tf)[:,1]
y_valid_preds = model.predict_proba(X_valid_tf)[:,1]
print('done')

In [ ]:
df_training_prob = pd.DataFrame([y_train[:10].values, y_train_preds[:10]]).transpose()
df_training_prob.columns = ['Actual', 'Probability']
df_training_prob = df_training_prob.astype({"Actual": int})

In [ ]:
df_training_prob

If we use L1 normalization, we get all probability values to be exactly 0.5. And if we use L2 normalization, all values are below 0.5, meaning all predictions point to 0. This is problematic, and probably indicates that there is not enough data to perform the training. I find this a bit surprising.

We can visualize the top positive and negative words predicting whether a patient is sick or not.

In [ ]:
def get_most_important_features(vectorizer, model, n=5):
    index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}
    
    # loop for each class
    classes ={}
    for class_index in range(model.coef_.shape[0]):
        word_importances = [(el, index_to_word[i]) for i,el in enumerate(model.coef_[class_index])]
        sorted_coeff = sorted(word_importances, key = lambda x : x[0], reverse=True)
        tops = sorted(sorted_coeff[:n], key = lambda x : x[0])
        bottom = sorted_coeff[-n:]
        classes[class_index] = {
            'tops':tops,
            'bottom':bottom
        }
    return classes

In [ ]:
def plot_important_words(top_scores, top_words, bottom_scores, bottom_words, name):
    y_pos = np.arange(len(top_words))
    top_pairs = [(a,b) for a,b in zip(top_words, top_scores)]
    top_pairs = sorted(top_pairs, key=lambda x: x[1])
    
    bottom_pairs = [(a,b) for a,b in zip(bottom_words, bottom_scores)]
    bottom_pairs = sorted(bottom_pairs, key=lambda x: x[1], reverse=True)
    
    top_words = [a[0] for a in top_pairs]
    top_scores = [a[1] for a in top_pairs]
    
    bottom_words = [a[0] for a in bottom_pairs]
    bottom_scores = [a[1] for a in bottom_pairs]
    
    fig = plt.figure(figsize=(10, 15))
    
    plt.subplot(121)
    plt.barh(y_pos,bottom_scores, align='center', alpha=0.5)
    plt.title('Negative', fontsize=20)
    plt.yticks(y_pos, bottom_words, fontsize=14)
    plt.suptitle('Key words', fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplot(122)
    plt.barh(y_pos,top_scores, align='center', alpha=0.5)
    plt.title('Positive', fontsize=20)
    plt.yticks(y_pos, top_words, fontsize=14)
    plt.suptitle(name, fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplots_adjust(wspace=0.8)
    plt.show()

In [ ]:
importance = get_most_important_features(my_vect, clf, 50)
top_scores = [a[0] for a in importance[0]['tops']]
top_words = [a[1] for a in importance[0]['tops']]
bottom_scores = [a[0] for a in importance[0]['bottom']]
bottom_words = [a[1] for a in importance[0]['bottom']]
plot_important_words(top_scores, top_words, bottom_scores, bottom_words, "Most important words")

It's very interesting to note that the unknown word 'xxxx' is such a strong negative indicator

There is no point trying to test the accuracy of the model yet since it is clearly terrible. It fails with both L1 and L2. We need to try another approach from what the author did. One possibility is instead to use tf-idf vectorization.

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score  

thresh = 0.46
X_test_tf = my_vect.transform(X_test.astype(str)) # Get the output labels as separate variables.
y_test_preds = model.predict_proba(X_test_tf)[:,1] # These will be probabiilities from logistic regression
assert len(y_test_preds) == len(y_test), 'different lengths!'

In [ ]:
y_preds = y_test_preds > thresh

In [ ]:
y_test_preds[:10]

In [ ]:
y_preds[:10]

In [ ]:
test_recall = recall_score(y_test, y_preds, thresh)
test_precision = precision_score(y_test, y_preds, thresh)

Unsurprisingly, the values don't work. Let's try TF-IDF embeddings.

In [ ]:
caml_notes_all.head()

In [ ]:
tfidf_caml = caml_notes_all[["LABEL", "TEXT"]].copy()
tfidf_caml.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1,3))

In [ ]:
X = tfidf_caml.TEXT
y = tfidf_caml.LABEL
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.20, random_state = 44)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size = 0.50, random_state = 44)

In [ ]:
# First train using the X_train vocabulary. You have to "unlist" each set, hence the lambda function.
tfidf_vectorizer.fit(X_train.apply(lambda x: ' '.join(x)))
len(tfidf_vectorizer.vocabulary_)

In [ ]:
X_train_tf = tfidf_vectorizer.transform(X_train.astype(str))
X_valid_tf = tfidf_vectorizer.transform(X_val.astype(str))

In [ ]:
clf = LogisticRegression(C = 0.0001, penalty = 'l2', random_state = 44)
clf.fit(X_train_tf, y_train)
model = clf
print('done')

In [ ]:
# We now calculate the probability of being sick based on the model
y_train_preds = model.predict_proba(X_train_tf)[:,1]
y_valid_preds = model.predict_proba(X_valid_tf)[:,1]
y_valid_preds[:10]

In [ ]:
df_training_prob = pd.DataFrame([y_train[:10].values, y_train_preds[:10]]).transpose()
df_training_prob.columns = ['Actual', 'Probability']
df_training_prob = df_training_prob.astype({"Actual": int})
print('done')

In [ ]:
over_5 = df_training_prob[['Probability']] > 0.47
over_5.sum(axis = 0)

We are still not getting any values over 0.5!!!! No way to get any precision measurements. Not sure what is going wrong.

In [ ]:
importance = get_most_important_features(tfidf_vectorizer, clf, 50)
top_scores = [a[0] for a in importance[0]['tops']]
top_words = [a[1] for a in importance[0]['tops']]
bottom_scores = [a[0] for a in importance[0]['bottom']]
bottom_words = [a[1] for a in importance[0]['bottom']]
plot_important_words(top_scores, top_words, bottom_scores, bottom_words, "Most important words")

One thing that's interesting is that with the new vectorization, the words and their importance are in a different order. I used n-grams to see if that might improve things.

In [ ]:
thresh = 0.49
X_test_tf = tfidf_vectorizer.transform(X_test.astype(str)) # Get the output labels as separate variables.
y_test_preds = model.predict_proba(X_test_tf)[:,1] # These will be probabiilities from logistic regression

In [ ]:
y_preds = y_test_preds > thresh
y_preds.sum()

In [ ]:
y_test_preds[:10]

## TRY COMPUTING LOSSES WITH THESE NEW LOG REG THRESHOLDS, AND DOCUMENT THE THRESHOLDS

## Conclusion

So it looks like with less than 4000 records, logistic regression is simply too weak a model to identify meaning predictions. There is a slight issue of prevalence, with the prevalence being less than 50%. We can try balancing he dataset and see if that makes any difference.

   ### STEPS

(1) Preprocess all the text, but leave each doc as string, not list

(2) Apply the preprocessing 

dtf["text_clean"] = dtf["text"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, 
          lst_stopwords=lst_stopwords))

(3) Split the data

(4) Create the vectorizers

#### Count (classic BoW)
vectorizer_bow = feature_extraction.text.CountVectorizer(max_features=10000, ngram_range=(1,2))

#### Tf-Idf (advanced variant of BoW)
vectorizer_tfidf = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

Fit the train data to the vectorizer
Transform corpus using vectorizer to produce X-train

corpus = dtf_train["text_clean"]
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

In [ ]:
# Create a classifier
classifier = naive_bayes.MultinomialNB()

In [ ]:
## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])## train classifier
model["classifier"].fit(X_train, y_train)## test
X_test = dtf_test["text_clean"].values
predicted = model.predict(X_test)
predicted_prob = model.predict_proba(X_test)

In [ ]:
classes = np.unique(y_test)
y_test_array = pd.get_dummies(y_test, drop_first=False).values
    ## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test, predicted)
auc = metrics.roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(metrics.classification_report(y_test, predicted))
    
## Plot confusion matrix
cm = metrics.confusion_matrix(y_test, predicted)
fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, 
            cbar=False)
ax.set(xlabel="Pred", ylabel="True", xticklabels=classes, 
       yticklabels=classes, title="Confusion matrix")
plt.yticks(rotation=0)
fig, ax = plt.subplots(nrows=1, ncols=2)
## Plot roc
for i in range(len(classes)):
    fpr, tpr, thresholds = metrics.roc_curve(y_test_array[:,i],  
                           predicted_prob[:,i])
    ax[0].plot(fpr, tpr, lw=3, 
              label='{0} (area={1:0.2f})'.format(classes[i], 
                              metrics.auc(fpr, tpr))
               )
ax[0].plot([0,1], [0,1], color='navy', lw=3, linestyle='--')
ax[0].set(xlim=[-0.05,1.0], ylim=[0.0,1.05], 
          xlabel='False Positive Rate', 
          ylabel="True Positive Rate (Recall)", 
          title="Receiver operating characteristic")
ax[0].legend(loc="lower right")
ax[0].grid(True)
    
## Plot precision-recall curve
for i in range(len(classes)):
    precision, recall, thresholds = metrics.precision_recall_curve(
                 y_test_array[:,i], predicted_prob[:,i])
    ax[1].plot(recall, precision, lw=3, 
               label='{0} (area={1:0.2f})'.format(classes[i], 
                                  metrics.auc(recall, precision))
              )
ax[1].set(xlim=[0.0,1.05], ylim=[0.0,1.05], xlabel='Recall', 
          ylabel="Precision", title="Precision-Recall curve")
ax[1].legend(loc="best")
ax[1].grid(True)
plt.show()

In [ ]:
# test_prevalence = calc_prevalence(y_test)
# auc_test = roc_auc_score(y_test, y_preds)

# print('Train prevalence(n = %d): %.3f'%(len(y_train), train_prevalence))
# print('Valid prevalence(n = %d): %.3f'%(len(y_valid), valid_prevalence))
# print('Test prevalence(n = %d): %.3f'%(len(y_test), test_prevalence))
# print('Train AUC:%.3f'%auc_train)
# print('Valid AUC:%.3f'%auc_valid)
# print('Valid AUC:%.3f'%auc_test)
# print('Train recall:%.3f'%train_recall)
# print('Valid recall:%.3f'%valid_recall)
# print('Test recall:%.3f'%test_recall)
# print('Train precision:%.3f'%train_precision)
# print('Valid precision:%.3f'%valid_precision)
# print('Test precision:%.3f'%test_precision)# Quickly throw metrics into a table format. It's easier to eyeball it for a side-by-side comparison.


In [ ]:
df_test_perf_metrics = pd.DataFrame([
    [train_prevalence, valid_prevalence, test_prevalence],
    [train_recall, valid_recall, test_recall], 
    [train_precision, valid_precision, test_precision],
    [auc_train, auc_valid, auc_test]], columns=['Training', 'Validation', 'Test'])df_test_perf_metrics.rename(index={0:'Prevalence', 1:'Recall', 2:'Precision', 3:'AUC'}, 
                 inplace=True)df_test_perf_metrics.round(3)

In [ ]:
## for word embedding
import gensim
import gensim.downloader as gensim_api